In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns, date_to_day_number
from settings import *


DATA_PATH: /Users/dan/ClimateIreland/CI-Status-Report/CI-Status-Report-Dash/data/


In [3]:
def percentile_series(df,col):
    """
    Returns series with percentile catagory based on column name submitted
    """
    def assign_percentile(df,row,col):
        val = row[col]
        if val >= np.percentile(df[col], 95):
            return 95
        elif val >= np.percentile(df[col], 75):
            return 75
        elif val >= np.percentile(df[col], 50):
            return 50
        elif val >= np.percentile(df[col], 25):
            return 25
        elif val >= np.percentile(df[col], 5):
            return 5
        else:
            return 0
    print('95 ' + str(np.percentile(df[col], 95)))
    print('75 ' + str(np.percentile(df[col], 75)))
    print('50 ' + str(np.percentile(df[col], 50)))
    print('25 ' + str(np.percentile(df[col], 25)))
    print('5 ' + str(np.percentile(df[col], 5)))
    return df.apply (lambda row: assign_percentile(df,row,col), axis=1)

In [ ]:
"""
Figure 3.10
"""
data_path = DATA_PATH+'Oceanic_Domain/3.5SeaState/Figure3.10/'
data_csv = data_path + 'Figure3.10_data.csv'

In [ ]:
"""
Tidy data for charts
"""


xls = pd.ExcelFile(
    data_path+'SeaState_M3.xlsx')
original_df = pd.read_excel(xls, 'M3',skiprows=1)
original_df


In [ ]:
columns_dict = {'datetime':'Dates',
               'mean__daily__sea_surface_wave_significant_height':'WaveHeight.1',
                  }
df_16 = map_columns(columns_dict, original_df)
df_16 = df_16[df_16['datetime'].notna()]
columns_dict = {'datetime':'Dates.1',
               'mean__daily__sea_surface_wave_significant_height':'WaveHeight.2',
                  }
df_17 = map_columns(columns_dict, original_df)
df_17 = df_17[df_17['datetime'].notna()]
columns_dict = {'datetime':'Dates.2',
               'mean__daily__sea_surface_wave_significant_height':'WaveHeight.3',
                  }
df_18 = map_columns(columns_dict, original_df)
df_18 = df_18[df_18['datetime'].notna()]
df = pd.concat([df_16,df_17,df_18])
df['location']='Buoy_M3'
# df.to_csv(data_csv) #commented out as the file should be kept as is

In [ ]:
data_path = DATA_PATH+'Oceanic_Domain/3.5SeaState/Figure3.10/'
data_csv = data_path + 'Figure3.10_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df['xAxis'] = df.apply (lambda row: date_to_day_number(row), axis=1)
df_16 = df.loc[(df['datetime'].dt.year == 2016)]
df_17 = df.loc[(df['datetime'].dt.year == 2017)]
df_18 = df.loc[(df['datetime'].dt.year == 2018)]
trace_16 = go.Scatter(x=df['xAxis'],
                      y=df_16['mean__daily__sea_surface_wave_significant_height'],
                         name='2016',
                         mode='markers+lines',
                         text=df['datetime'],
                         marker=dict(color=TIMESERIES_COLOR_1,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_1,
                                      width=1),
                         hovertemplate='%{text|%d-%b}-2016<br>' +
                         '<b>2016 Daily Average</b><br>' +
                         'Wave Height: %{y:.2f} m<br>' +
                         '<extra></extra>'
                         )
trace_17 = go.Scatter(x=df['xAxis'],
                      y=df_17['mean__daily__sea_surface_wave_significant_height'],
                         name='2017',
                         mode='markers+lines',
                         text=df['datetime'],
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{text|%d-%b}-2017<br>' +
                         '<b>2017 Daily Average</b><br>' +
                         'Wave Height: %{y:.2f} m<br>' +
                         '<extra></extra>'
                         )
trace_18 = go.Scatter(x=df['xAxis'],
                      y=df_18['mean__daily__sea_surface_wave_significant_height'],
                         name='2018',
                         mode='markers+lines',
                         text=df['datetime'],
                         marker=dict(color=TIMESERIES_COLOR_3,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_3,
                                      width=1),
                         hovertemplate='%{text|%d-%b}-2018<br>' +
                         '<b>2018 Daily Average</b><br>' +
                         'Wave Height: %{y:.2f} m<br>' +
                         '<extra></extra>'
                         )
figure_3_10 = go.Figure(data=[trace_16,trace_17,trace_18], layout=TIMESERIES_LAYOUT)
figure_3_10.update_layout(
    yaxis=dict(title='Mean Significant Wave Height, H<sub>s</sub> (m)'),
    xaxis=dict(
        title="Month",
        ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
        showgrid=False,
        tickvals=[50,80, 110, 140, 170, 200, 230, 260, 290, 320, 350, 380],
    )
)
figure_3_10

In [4]:
"""
Figure 3.11
"""
data_path = DATA_PATH+'Oceanic_Domain/3.5SeaState/Figure3.11/'
data_csv = data_path + 'Figure3.11_data.csv'

In [34]:
def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale  



In [5]:
"""
Tidy data for charts 
"""


xls = pd.ExcelFile(
    data_path+'Figure3.11_HeatMaps_SignificantWaveHeight.xlsx')
original_df_m2 = pd.read_excel(xls, 'M2',skiprows=2)
original_df_m2

,Unnamed: 0,UTC,Unnamed: 2,m,s,degrees_true,m.1,Unnamed: 7,Jan,2002-01-01 00:00:00,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,M2,2001-05-03T14:00:00Z,2001-05-03,1.1,4.0,NaN,NaN,NaN,Jan,2002-01-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M2,2001-05-03T15:00:00Z,2001-05-03,1.0,4.0,NaN,NaN,NaN,Jan,2002-01-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M2,2001-05-03T16:00:00Z,2001-05-03,1.0,5.0,NaN,NaN,NaN,Jan,2002-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M2,2001-05-03T17:00:00Z,2001-05-03,0.8,5.0,NaN,NaN,NaN,Jan,2002-01-05,...,NaN,0.731608,1.128327,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2,2001-05-03T18:00:00Z,2001-05-03,0.8,5.0,NaN,NaN,NaN,Jan,2002-01-06,...,0.835445,1.162483,1.669365,1.846567,1.389709,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131839,NaN,2018-12-31T20:00:00Z,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131840,NaN,2018-12-31T21:00:00Z,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131841,NaN,2018-12-31T22:00:00Z,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131842,NaN,2018-12-31T23:00:00Z,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
original_df_m3 = pd.read_excel(xls, 'M3',skiprows=2)
original_df_m3

,Unnamed: 0,UTC,Unnamed: 2,m,s,degrees_true,m.1,Unnamed: 7,Jan,2002-01-01 00:00:00,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,M3,2002-07-22T14:00:00Z,2002-07-22,NaN,NaN,NaN,NaN,NaN,Jan,2002-01-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M3,2002-07-22T15:00:00Z,2002-07-22,NaN,NaN,NaN,NaN,NaN,Jan,2002-01-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M3,2002-07-22T16:00:00Z,2002-07-22,NaN,NaN,NaN,NaN,NaN,Jan,2002-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M3,2002-07-22T17:00:00Z,2002-07-22,NaN,NaN,NaN,NaN,NaN,Jan,2002-01-05,...,2.287617,2.806179,3.638651,4.463836,3.659728,NaN,NaN,NaN,NaN,NaN
4,M3,2002-07-22T18:00:00Z,2002-07-22,NaN,NaN,NaN,NaN,NaN,Jan,2002-01-06,...,NaN,NaN,3.606742,2.899549,4.513961,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115125,M3,2018-12-31T20:00:00Z,2018-12-31,2.656,7.969,253.0,3.984,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115126,M3,2018-12-31T21:00:00Z,2018-12-31,2.813,8.203,252.0,4.219,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115127,M3,2018-12-31T22:00:00Z,2018-12-31,2.656,7.734,252.0,3.984,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115128,M3,2018-12-31T23:00:00Z,2018-12-31,2.500,7.383,250.0,3.281,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
columns_dict = {'datetime':'UTC',
               'sea_surface_wave_significant_height':'m',
                  }
filtered_df_m2 = map_columns(columns_dict, original_df_m2)

filtered_df_m2['datetime'] = pd.to_datetime(filtered_df_m2['datetime'])
filtered_df_m2 = filtered_df_m2.drop(filtered_df_m2[filtered_df_m2.datetime.dt.year < 2002].index)
filtered_df_m2 = filtered_df_m2.drop(filtered_df_m2[filtered_df_m2.datetime.dt.year > 2018].index)
# Create series with datetime as index
series =pd.Series(filtered_df_m2['sea_surface_wave_significant_height'].values,index=filtered_df_m2['datetime'])
# Calc mean monthly 
series_mean_monthly = series.resample('M').mean()
df_m2 = pd.DataFrame()
df_m2['mean__monthly__sea_surface_wave_significant_height'] = series_mean_monthly
df_m2['datetime'] = df_m2.index
df_m2.reset_index(drop=True, inplace=True)
df_m2 = df_m2[['datetime','mean__monthly__sea_surface_wave_significant_height']]
df_m2['location']='Buoy_M2'
df_m2


filtered_df_m3 = map_columns(columns_dict, original_df_m3)

filtered_df_m3['datetime'] = pd.to_datetime(filtered_df_m3['datetime'])
filtered_df_m3 = filtered_df_m3.drop(filtered_df_m3[filtered_df_m3.datetime.dt.year < 2002].index)
filtered_df_m3 = filtered_df_m3.drop(filtered_df_m3[filtered_df_m3.datetime.dt.year > 2018].index)
# Create series with datetime as index
series =pd.Series(filtered_df_m3['sea_surface_wave_significant_height'].values,index=filtered_df_m3['datetime'])
# Calc mean monthly 
series_mean_monthly = series.resample('M').mean()
df_m3 = pd.DataFrame()
df_m3['mean__monthly__sea_surface_wave_significant_height'] = series_mean_monthly
df_m3['datetime'] = df_m3.index
df_m3.reset_index(drop=True, inplace=True)
df_m3 = df_m3[['datetime','mean__monthly__sea_surface_wave_significant_height']]
df_m3['location']='Buoy_M3'
df = pd.concat([df_m2,df_m3])
# df.to_csv(data_csv) #commented out as the file should be kept as is

In [72]:
df


,datetime,mean__monthly__sea_surface_wave_significant_height,location
0,2002-01-31 00:00:00+00:00,1.841497,Buoy_M2
1,2002-02-28 00:00:00+00:00,1.981915,Buoy_M2
2,2002-03-31 00:00:00+00:00,1.339107,Buoy_M2
3,2002-04-30 00:00:00+00:00,1.245480,Buoy_M2
4,2002-05-31 00:00:00+00:00,1.209447,Buoy_M2
...,...,...,...
193,2018-08-31 00:00:00+00:00,2.125110,Buoy_M3
194,2018-09-30 00:00:00+00:00,2.541330,Buoy_M3
195,2018-10-31 00:00:00+00:00,3.115716,Buoy_M3
196,2018-11-30 00:00:00+00:00,4.216892,Buoy_M3


In [75]:
data_path = DATA_PATH+'Oceanic_Domain/3.5SeaState/Figure3.11/'
data_csv = data_path + 'Figure3.11_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[df['mean__monthly__sea_surface_wave_significant_height'].notna()]
df_min = df['mean__monthly__sea_surface_wave_significant_height'].min()
df_max = df['mean__monthly__sea_surface_wave_significant_height'].max()
df['percentile'] = percentile_series(df,'mean__monthly__sea_surface_wave_significant_height')
m2_df = df.loc[(df['location'] == 'Buoy_M2')]
percentile_series(m2_df,'mean__monthly__sea_surface_wave_significant_height')
m3_df = df.loc[(df['location'] == 'Buoy_M3')]
# df.to_csv(data_csv)

95 4.330077403414208
75 2.6295735900962884
50 1.742650231124807
25 1.148873239436618
5 0.7447525524992615
95 1.848626547340247
75 1.4679972013743565
50 1.1580443180635323
25 0.8539891789943962
5 0.6697207661290318


In [76]:
df_max

6.016856459330146

In [77]:
# bvals = [0.5,0.7, 0.85, 1.15, 1.46, 1.85,2]
# bvals = [df_min,0.7, 1.1, 1.8, 2.7, 4.3,df_max]
# bvals = [df_min,0.7, 1.1, 1.8, 2.7, 4.3,df_max]
bvals = [df_min,0.7, 1.1, 1.8, 2.6, 4.3,df_max]

colors = ['rgb(98, 55, 155)', 'rgb(184, 197, 229)', 'rgb(166, 206, 93)' , 'rgb(255, 254, 66)', 'rgb(239, 191, 49)','rgb(219, 32, 1)']

dcolorsc = discrete_colorscale(bvals, colors)
dcolorsc

colorscale=[
    [0, "rgb(255, 255, 255)"],
    [0.1, "rgb(255, 255, 255)"],
    [0.1, "rgb(180, 180, 0)"], #here is where you set your min color
    [0.9, "rgb(0, 0, 180)"], #here is where you set your max color
    [0.9, "rgb(255, 255, 255)"],
    [1, "rgb(255, 255, 255)"],
]

trace_m2 = go.Heatmap(
    z=m2_df['mean__monthly__sea_surface_wave_significant_height'],
    x=m2_df['datetime'].dt.month,
    y=m2_df['datetime'].dt.year,
    text=m2_df['mean__monthly__sea_surface_wave_significant_height'],
    colorscale=dcolorsc, 
    zmin=df_min,
    zmax=df_max,
    colorbar=dict(
                tickmode='array',
                thickness=10,
                len=0.9,
        
    title='<b>Percentile</b> (m)',
            ticktext=[
                '<b>Min.</b> (0.5)', 
                '<b> 5%</b> (0.7)', 
                '<b>25%</b> (1.1)', 
                '<b>50%</b> (1.8)',
                '<b>75%</b> (2.7)', 
                '<b>95%</b> (4.3)'],
            tickvals=[0,5,24,47, 71, 90]  
    ),
    hovertemplate='%{x} %{y}<br>'+
    'Wave Height: %{text:.2f} m<extra></extra>'
)
figure_3_11_a = go.Figure(data=trace_m2, layout=TIMESERIES_LAYOUT)
figure_3_11_a.update_layout(
yaxis=dict(
    title='Year',
    nticks=12),
xaxis=dict(
    title='Month',
    ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
    showgrid=False,
   tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
))

# Include white lines as xaxis gridlines
x0=1.5
for i in range(0,11):
    figure_3_11_a.add_shape(type='line',
                        x0=x0, y0=2001.5, x1=x0, y1=2019,
                        line=dict(color='White', width=3))
    x0+=1

figure_3_11_a

In [78]:
trace_m3 = go.Heatmap(
    z=m3_df['mean__monthly__sea_surface_wave_significant_height'],
    x=m3_df['datetime'].dt.month,
    y=m3_df['datetime'].dt.year,
    text=m3_df['mean__monthly__sea_surface_wave_significant_height'],
    colorscale=dcolorsc, 
    zmin=df_min,
    zmax=df_max,
    colorbar=dict(
                tickmode='array',
                thickness=10,
                len=0.9,
        
    title='<b>Percentile</b> (m)',
            ticktext=[
                '<b>Min.</b> (0.5)', 
                '<b> 5%</b> (0.7)', 
                '<b>25%</b> (1.1)', 
                '<b>50%</b> (1.8)',
                '<b>75%</b> (2.7)', 
                '<b>95%</b> (4.3)'],
            tickvals=[0,5,24,47, 71, 90]  
    ),
    hovertemplate='%{x} %{y}<br>'+
    'Wave Height: %{text:.2f} m<extra></extra>'
)
figure_3_11_b = go.Figure(data=trace_m3, layout=TIMESERIES_LAYOUT)
figure_3_11_b.update_layout(
yaxis=dict(
    title='Year',
    nticks=12),
xaxis=dict(
    title='Month',
    ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
    showgrid=False,
   tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
))

# Include white lines as xaxis gridlines
x0=1.5
for i in range(0,11):
    figure_3_11_a.add_shape(type='line',
                        x0=x0, y0=2001.5, x1=x0, y1=2019,
                        line=dict(color='White', width=3))
    x0+=1

figure_3_11_b